## PIL TEST - IMPLEMENTATION

Here we will see the implementation of the [PIL Test](Fairness-New-Definitions.ipynb) on the [Adult Dataset](https://archive.ics.uci.edu/ml/datasets/adult).

### IMPORTNG THE NECESSARY LIBRARIES

We will be using the open-source [AIF360](https://github.com/Trusted-AI/AIF360) package to use several fairness based metrics.

In [7]:
import numpy as np
np.set_printoptions(suppress = True)
import pandas as pd

import matplotlib.pyplot as plt

# Importing the Dataset
from aif360.datasets import AdultDataset
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult

from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric

from aif360.metrics.utils import compute_boolean_conditioning_vector
from common_utils import compute_metrics

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn

import pickle

### DATASET

In [2]:
priv_group = [{'sex':1}]
unpriv_group = [{'sex':0}]

In [3]:
data_adult = load_preproc_data_adult(['sex'])

### LOADING PRE-TRAINED MODELS

We trained two models simple logistic regression and equalized odds regularized logistic regression [here](Fairness-Regularization.ipynb). So, we load these models for testing our new definition of fairness.

In [34]:
class Log_Reg(nn.Module):
    def __init__(self, size_in):
        super().__init__()
        self.linear = nn.Linear(size_in, 1)
    def forward(self, x):
        prob_pred = torch.sigmoid(self.linear(x))
        return prob_pred

In [35]:
with open("raw-test-data.bin", "rb") as input:
    dset_raw_tst = pickle.load(input)
    
with open("trained-std-scaler.bin", "rb") as input:
    scaler = pickle.load(input)

In [36]:
M = Log_Reg(len(dset_raw_tst.feature_names)) # Non-Fairness Based Regularized Model
M_F = Log_Reg(len(dset_raw_tst.feature_names)) # Fairness Based Regularization Model

In [37]:
M.load_state_dict(torch.load("simple-logistic-regression.pt"))
M_F.load_state_dict(torch.load("equalized-odd-regualarized-logistic-regression.pt"))

M.eval()
M_F.eval()

Log_Reg(
  (linear): Linear(in_features=18, out_features=1, bias=True)
)

### PREDICTIONS OF THE MODELS

Here we get the predicted values of both the models.

In [71]:
dset_tst_pred_M = dset_raw_tst.copy(deepcopy=True)
dset_tst_pred_M_F = dset_raw_tst.copy(deepcopy=True)
dset_tst = scaler.transform(dset_tst_pred.features)
dset_tst_pred_M.labels = (M(torch.from_numpy(dset_tst).float()) > 0.5).numpy().astype(float)
dset_tst_pred_M_F.labels = (M_F(torch.from_numpy(dset_tst).float()) > 0.5).numpy().astype(float)

We now prepare the data for training the 'adversarial' models for predicting the protected group membership from the remaining features of the test data (and the predicted labels of the original model), as per the [PIL Test](Fairness-New-Definitions.ipynb).

In [72]:
y = dset_tst_pred_M.features[:,1].reshape((-1, 1))
# replacing the gender column with model M's predicted values
dset_tst_pred_M.features[:,1] = dset_tst_pred_M.labels.ravel()
# replacing the gender column with model M_F's predicted values
dset_tst_pred_M_F.features[:,1] = dset_tst_pred_M_F.labels.ravel()

dset_other_features = dset_raw_tst.features[:,[0]+list(range(2,len(dset_tst_pred_M.feature_names)))]

In [73]:
scaler_M = StandardScaler()
dset_tst_M = scaler_M.fit_transform(dset_tst_pred_M.features)

scaler_M_F = StandardScaler()
dset_tst_M_F = scaler_M_F.fit_transform(dset_tst_pred_M_F.features)

In [74]:
dset_tst_M = torch.from_numpy(dset_tst_M).float()
dset_tst_M_F = torch.from_numpy(dset_tst_M_F).float()
dset_other_features = torch.from_numpy(dset_other_features).float()
y = torch.from_numpy(y).float()

In [75]:
adv_mod_pred_M = Log_Reg(len(dset_tst_pred_M.feature_names))
adv_mod_wo_pred = Log_Reg((len(dset_tst_pred_M.feature_names)-1))
adv_mod_pred_M_F = Log_Reg(len(dset_tst_pred_M_F.feature_names))

In [76]:
num_epochs = 5000 # Number of Epochs
learning_rate = 0.01 # Learning Rate

# Stochastic Gradient Descent Optimizers
optimizer_M = torch.optim.SGD(adv_mod_pred_M.parameters(), lr= learning_rate)
optimizer_M_F = torch.optim.SGD(adv_mod_pred_M_F.parameters(), lr= learning_rate)
optimizer_wo_pred = torch.optim.SGD(adv_mod_wo_pred.parameters(), lr= learning_rate)

# Binary Cross Entropy Loss Functions
criterion = nn.BCELoss()

In [77]:
# Training adv_mod_wo_pred
print("Training adv_mod_wo_pred:")
adv_mod_wo_pred.train()
for epoch in range(num_epochs):
    p_pred = adv_mod_wo_pred(dset_other_features)
    loss= criterion(p_pred, y)
    
    loss.backward()
    optimizer_wo_pred.step()
    
    optimizer_wo_pred.zero_grad()
    
    if (epoch+1) % 500== 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')
        
print("Trained adv_mod_wo_pred's Performance on Data:")        
with torch.no_grad():
    y_pred = (p_pred > 0.5).numpy().astype(float)
    accuracy = (np.sum(np.sum(np.array(y_pred.ravel()) == np.array(y.ravel())))/len(y.ravel()))*100
    print("Accuracy: ", accuracy)
    

Training adv_mod_wo_pred:
epoch: 500, loss = 0.3487
epoch: 1000, loss = 0.2993
epoch: 1500, loss = 0.2686
epoch: 2000, loss = 0.2466
epoch: 2500, loss = 0.2302
epoch: 3000, loss = 0.2177
epoch: 3500, loss = 0.2079
epoch: 4000, loss = 0.1999
epoch: 4500, loss = 0.1933
epoch: 5000, loss = 0.1878
Trained adv_mod_wo_pred's Performance on Data:
Accuracy:  91.34648194908893


In [78]:
# Training adv_mod_pred_M
print("Training adv_mod_pred_M:")
adv_mod_pred_M.train()
for epoch in range(num_epochs):
    p_pred = adv_mod_pred_M(dset_tst_M)
    loss= criterion(p_pred, y)
    
    loss.backward()
    optimizer_M.step()
    
    optimizer_M.zero_grad()
    
    if (epoch+1) % 500== 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')
        
print("Trained adv_mod_pred_M's Performance on Data:")        
with torch.no_grad():
    y_pred = (p_pred > 0.5).numpy().astype(float)
    accuracy = (np.sum(np.sum(np.array(y_pred.ravel()) == np.array(y.ravel())))/len(y.ravel()))*100
    print("Accuracy: ", accuracy)

Training adv_mod_pred_M:
epoch: 500, loss = 0.2004
epoch: 1000, loss = 0.1083
epoch: 1500, loss = 0.0729
epoch: 2000, loss = 0.0545
epoch: 2500, loss = 0.0434
epoch: 3000, loss = 0.0360
epoch: 3500, loss = 0.0307
epoch: 4000, loss = 0.0268
epoch: 4500, loss = 0.0237
epoch: 5000, loss = 0.0213
Trained adv_mod_pred_M's Performance on Data:
Accuracy:  100.0


In [79]:
# Training adv_mod_pred_M
print("Training adv_mod_pred_M_F:")
adv_mod_pred_M_F.train()
for epoch in range(num_epochs):
    p_pred = adv_mod_pred_M(dset_tst_M_F)
    loss= criterion(p_pred, y)
    
    loss.backward()
    optimizer_M_F.step()
    
    optimizer_M_F.zero_grad()
    
    if (epoch+1) % 500== 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')
        
print("Trained adv_mod_pred_M_F's Performance on Data:")        
with torch.no_grad():
    y_pred = (p_pred > 0.5).numpy().astype(float)
    accuracy = (np.sum(np.sum(np.array(y_pred.ravel()) == np.array(y.ravel())))/len(y.ravel()))*100
    print("Accuracy: ", accuracy)

Training adv_mod_pred_M_F:
epoch: 500, loss = 0.1881
epoch: 1000, loss = 0.1881
epoch: 1500, loss = 0.1881
epoch: 2000, loss = 0.1881
epoch: 2500, loss = 0.1881
epoch: 3000, loss = 0.1881
epoch: 3500, loss = 0.1881
epoch: 4000, loss = 0.1881
epoch: 4500, loss = 0.1881
epoch: 5000, loss = 0.1881
Trained adv_mod_pred_M_F's Performance on Data:
Accuracy:  94.91571691803726
